In [1]:
import os
import json

from image_utils import *

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Data

In [2]:
IMAGE_DIR = 'data/train/'
with open('./data/train/annotations.json') as json_data:
    data = json.load(json_data)

## Generators

### Load Data

In [3]:
from whale_mask_generator import WhaleMaskGenerator

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
train_generator = WhaleMaskGenerator(IMAGE_DIR, data, 4)

# Test Runner

In [5]:
def run_test(H, W, model):
    pass

# Training

In [6]:
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPool2D, Conv2DTranspose
from keras.layers import Activation, BatchNormalization, concatenate
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, Callback
from keras import applications
import keras.backend as K
import tensorflow as tf

from tensorboard_logger import TensorBoardLogger

# Losses and Metrics

In [7]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jaccard(y_true, y_pred, threshold=0.5, axis=(1, 2, 3), smooth=1e-5): 
        pre = tf.cast(y_pred > threshold, dtype=tf.float32)
        truth = tf.cast(y_true > threshold, dtype=tf.float32)
        inse = tf.reduce_sum(tf.multiply(pre, truth), axis=axis)  # AND
        union = tf.reduce_sum(tf.cast(tf.add(pre, truth) >= 1, dtype=tf.float32), axis=axis)  # OR
        batch_iou = (inse + smooth) / (union + smooth)
        iou = tf.reduce_mean(batch_iou)
        return iou

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def jacckard_loss(y_true, y_pred):
    return 1 - jaccard(y_true, y_pred)

## SimpleNet

In [14]:
def create_simplenet(H, W, C):
    def conv_block(in_layer, filters, activation='relu'):
        x = Conv2D(filters, kernel_size=3, activation=activation, padding='same')(in_layer)
        x = BatchNormalization()(x)
        return x

    inputs = Input((H, W, C))
    
    x = conv_block(inputs, 32)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    x = conv_block(x, 256)
    x = conv_block(x, 256)
    
    out = Conv2D(4, kernel_size=3, activation='sigmoid', padding='same')(x)
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef, jaccard, 'accuracy'])

    return model

### Find Learning Rate

In [15]:
H, W = 224, 224

In [ ]:
model = create_simplenet(H, W, 1)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 224, 224, 32)      320       
_________________________________________________________________
batch_normalization_19 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 224, 224, 32)      9248      
_________________________________________________________________
batch_normalization_20 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 224, 224, 64)      18496     
_________________________________________________________________
batch_normalization_21 (Batc (None, 224, 224, 64)      256       
__________

In [ ]:
lrs = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
for lr in lrs:
    print(f'Learning Rate: {lr}')
    K.set_value(model.optimizer.lr, lr)
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=1
    )

Learning Rate: 1e-06
Epoch 1/1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


### Train!

In [ ]:
model = create_simplenet(H, W, 3)
tensorboard = TensorBoardLogger(log_name='Simple-64')

In [ ]:
# model.load_weights('checkpoints/simplenet.h5')

In [ ]:
fit_args = dict(
    generator=train_generator,
    steps_per_epoch=200,
    callbacks=[tensorboard],
    validation_data=val_generator,
    validation_steps=100,
    workers=6,
    epochs=10,
)

In [ ]:
K.set_value(model.optimizer.lr, 1e-5)

In [ ]:
fit_args['epochs'] = 2
model.fit_generator(**fit_args)

In [ ]:
model.save_weights('checkpoints/simplenet.h5')

In [ ]:
run_test(H, W, model)

## Pre-Trained Model

In [ ]:
def create_pretrained_model(base='ResNet50'):
    def upsample(in_layer, nout):
        x = Conv2DTranspose(nout, kernel_size=2, strides=2, activation='relu')(in_layer)
        x = BatchNormalization()(x)
        return x
    
    if base == 'VGG16':
        base = applications.vgg16.VGG16(
                include_top=False, 
                weights='imagenet', 
                pooling=None,
                input_shape=(H, W, 3),)
        for layer in base.layers[:5]:
            layer.trainable = False
    elif base == 'ResNet50':
        base = applications.resnet50.ResNet50(
                include_top=False, 
                weights='imagenet', 
                pooling=None,
                input_shape=(H, W, 3),)
        for layer in base.layers:
            layer.trainable = False
#         base.layers.pop()
#         base.output = [base.layers[-1].output]
    
    x = base.layers[-3].output
    x = upsample(x, 256)
    x = upsample(x, 256)
    x = upsample(x, 256)
#     x = upsample(x, 256)
    out = Conv2DTranspose(1, kernel_size=2, strides=2, activation='sigmoid')(x)
    
    model = Model(inputs=base.input, outputs=out)
    
    optimizer = Adam(lr=1e-4)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[dice_coef, jaccard, 'accuracy'])
    
    return model

In [ ]:
BS, H, W = 8, 224, 224
train_generator, val_generator = dl.get_train_val_generators(batch_size=BS, height=H, width=W)

In [ ]:
model = create_pretrained_model(base='VGG16')
model.summary()

In [ ]:
lrs = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
for lr in lrs:
    print(f'Learning Rate: {lr}')
    K.set_value(model.optimizer.lr, lr)
    model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=1
    )

In [ ]:
model = create_pretrained_model(base='VGG16')
tensorboard = TensorBoardLogger(log_name='VGG-224')

In [ ]:
model.load_weights('checkpoints/VGG-224')

In [ ]:
fit_args = dict(
    generator=train_generator,
    steps_per_epoch=200,
    callbacks=[tensorboard],
    validation_data=val_generator,
    validation_steps=100,
    workers=6,
    epochs=2,
)

In [ ]:
K.set_value(model.optimizer.lr, 1e-5)

In [ ]:
model.fit_generator(**fit_args)

In [ ]:
model.save_weights('checkpoints/VGG-224')

In [ ]:
run_test(H, W, model)

## UNET

In [ ]:
def get_unet(H, W, C):
    def conv_block(in_layer, filters, pooling=True):
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(in_layer)
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        mp = MaxPool2D(pool_size=(2, 2))(x) if pooling else None
        return x, mp
    
    def up_block(in_layer, conv_layer, filters):
        x = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(in_layer)
        x = concatenate([x, conv_layer], axis=3)
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        return x
    
    inputs = Input((H, W, C))
    
    c1, mp = conv_block(inputs, 32)
    c2, mp = conv_block(mp, 64)
    c3, mp = conv_block(mp, 128)
    c4, mp = conv_block(mp, 256)
    c5, _ = conv_block(mp, 512, pooling=False)
    
    x = up_block(c5, c4, 256)
    x = up_block(x, c3, 128)
    x = up_block(x, c2, 64)
    x = up_block(x, c1, 32)
    
    out = Conv2D(1, (1, 1), activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef, jaccard, 'accuracy'])

    return model

In [ ]:
BS, H, W = 16, 64, 64
train_generator, val_generator = dl.get_train_val_generators(batch_size=BS, height=H, width=W)

In [ ]:
model = get_unet(H, W, 3)
model.summary()

In [ ]:
lrs = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
for lr in lrs:
    print(f'Learning Rate: {lr}')
    K.set_value(model.optimizer.lr, lr)
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=1
    )

In [ ]:
model = get_unet(H, W, 3)
tensorboard = TensorBoardLogger(log_name='UNET-64')

In [ ]:
model.load_weights('checkpoints/unet-64.h5')

In [ ]:
K.set_value(model.optimizer.lr, 2e-5)

In [ ]:
fit_args = dict(
    generator=train_generator,
    steps_per_epoch=200,
    callbacks=[tensorboard],
    validation_data=val_generator,
    validation_steps=100,
    workers=6,
    epochs=2,
)

In [ ]:
model.fit_generator(**fit_args)

In [ ]:
model.save_weights('checkpoints/unet-64.h5')

In [ ]:
run_test(H, W, model)